# Aim

Attempting to implement the Support Vector Kalman method, as described at http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.672.2696&rep=rep1&type=pdf.


In [ ]:
from sklearn import svm
from sklearn.model_selection import KFold
import numpy as np
from aberrations import make_vibe_params, make_vibe_data, make_noisy_data, measurement_noise
from observer import make_kfilter, vibe_fit_freq, get_psd, predict, update, noise_filter
from matplotlib import pyplot as plt
from copy import deepcopy
%matplotlib inline

In [ ]:
from aberrations import time_id, f_sampling, f_1, times
W = 10
N = 10
steps = int(f_sampling * time_id)
times = np.arange(0, time_id, 1 / f_sampling)

In [ ]:
# set up state training data
truth = make_vibe_data(times=times,N=N)
measurements = make_noisy_data(truth)

In [ ]:
# first, let's run the Kalman filter for half a second normally
state, A, P, Q, H, R = make_kfilter(*vibe_fit_freq(noise_filter(get_psd(measurements)), N=N))
states_freq = np.zeros((steps, state.size))
for k in range(steps):
    state, P = update(H, P, R, state, measurements[k])
    states_freq[k] = state
    state, P = predict(A, P, Q, state)
    
pos_freq = np.array([H.dot(state) for state in states_freq]).flatten()

plt.figure(figsize=(10,10))
plt.plot(times, truth, label='Truth')
plt.plot(times, pos_freq, label='Freq filter')
plt.legend()

In [ ]:
def make_svk_state_transition(x_training, y_training):
    # normalize and rescale
    x_recentering = (np.min(x_training, axis=0) + np.max(x_training, axis=0))/2
    y_recentering = (np.min(y_training, axis=1) + np.max(y_training, axis=1))/2
    x_training -= np.tile(x_recentering, (W+1, 1))
    y_training -= np.repeat(y_recentering, W+1).reshape(state.size,W+1)
    x_scales = (np.max(x_training, axis=0) - np.min(x_training, axis=0))/2
    y_scales = (np.max(y_training, axis=1) - np.min(y_training, axis=1))/2
    x_training /= np.tile(x_scales, (W+1, 1))
    y_training /= np.repeat(y_scales, W+1).reshape(state.size,W+1)
    A = np.zeros((STATE_SIZE, STATE_SIZE))
    svk = [svm.SVR(kernel='linear',gamma='scale', C=1, epsilon=0.2)] * N
    for i in range(N):
        # help out the SVK by telling it the only state elements that matter
        # we know this is the case because we ran in frequency mode for half a second
        x_training_i = x_training[:,2*i:2*i+2]
        svk[i].fit(x_training_i, y_training[2*i])
        predictions = np.array([svk[i].predict(np.array([[1, 0]]))[0], 
                                svk[i].predict(np.array([[0, 1]]))[0]])
        A[2*i][2*i], A[2*i][2*i+1] = predictions * (y_scales/x_scales)[2*i:2*i+2]
        A[2*i+1][2*i] = 1
    return A

def make_svk_process_covariance(x_training, y_training):
    kfold = KFold(W)

In [ ]:
# evolve states based on this A. Note that the SVRs are getting trained but A itself isn't. May add KF for A later.
refresh_frame = 300
state = states_freq[refresh_frame]
states_svk = np.zeros(states_freq.shape)
states_svk[:refresh_frame] = states_freq[:refresh_frame]

In [ ]:
#plt.plot(times, np.array([H.dot(state) for state in states_svk]).flatten())

In [ ]:
STATE_SIZE = 2*N
k = refresh_frame
while k < steps:
    # Assumed k > W + 1, and theoretically we need at least k = 2/f_1 (though apparently 1/f_1 is working fine)
    if k == refresh_frame:
        print("Refreshing the model at timestep", k)
        x_training = deepcopy(states_svk[k-W-2:k-1]) # a row is the state at a specific timestep: size (W+1, state.size)
        y_training = deepcopy(states_svk[k-W-1:k].T) # a row is a state variable over the W timesteps: size (state.size, W+1)
        A = make_svk_state_transition(x_training, y_training)
    state, P = update(H, P, R, state, measurements[k])
    states_svk[k] = state
    state, P = predict(A, P, Q, state)
    k += 1

#plt.plot(times, np.array([H.dot(state) for state in states_svk]).flatten())

In [ ]:
pos_svk = np.array([H.dot(state) for state in states_svk]).flatten()

In [ ]:
plt.figure(figsize=(10,10))
lim = np.max(np.abs(truth))
plt.plot(times, truth, label='Truth')
plt.plot(times, pos_svk, label='SVK')
plt.legend()
plt.ylim(-lim, lim)

In [ ]:
plt.figure(figsize=(10,10))
res_svk = np.abs(pos_svk - truth)
res_freq = np.abs(pos_freq - truth)
plt.plot(times, res_freq, label='Freq residual: avg = ' 
         + str(np.round(np.mean(res_freq[refresh_frame:]), 3)))
plt.plot(times, res_svk, label='SVK residual: avg = ' 
         + str(np.round(np.mean(res_svk[refresh_frame:]), 3)))
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Residual (mas)")
plt.title("Residuals of frequency fit and Support Vector Kalman")
plt.ylim(0, 1)